In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 49.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8c35e7417853bf6f2a42574b6f0dbaa6c4978d14a62b02b54c24bda967b77f89
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


# Узнаем кол-во потоков

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

# Настройка SparkSession
Основные параметры:

 * -num-executors (spark.executor.instances) — количество экзекьютеров, 
 * --executor-memory (spark.executor.memory) — количество памяти у экзекьютера,
 * --execuor-cores (park.executor.cores) — количество ядер у экзекьютера.

На каждом сервере исполняется ОС, Hadoop/Yarn/OS Deamons, HDFS. Часть ресурсов потребляют они. HDFS (и другие файловые системы) имеют свою пропускную способность. Так же нужно учитывать сборщик мусора.

Есть две крайности: 1 ядро - 1 таск, 1 нода - 1 таск. Чаще всего оба варианта не самые удачные.

Рекомендация: не больше 5 ядер на 1 экзекьютор из-за пропуской способности HDFS.

In [ ]:
62 / ((16 - 1) / 3) / 1.1

11.272727272727272

## executor-memory
![Image of Yaktocat](https://miro.medium.com/max/1400/1*8yviZQXq9rXgBoup8W-kHA.png)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[2]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', f'{int(120000 / 2 /1.1)}mb')\
        .config('spark.executor.cores', 1)\
        .getOrCreate()

sc = spark.sparkContext

In [ ]:
spark

# Пример из презентации

In [ ]:
rdd = sc.parallelize(['After all is', 'said and', 'done more is', 'said than done'])

In [ ]:
def lower(row):
    return row.upper()

def lower(element1, element2):
    return element1 + element2

lower = lambda row: row.lower()

2

rdd.map(lambda row: row.lower()).collect()

['after all is', 'said and', 'done more is', 'said than done']

In [ ]:
rdd.take(1)

['After all is']

In [ ]:
rdd = sc.parallelize(['After all is', 'said and', 'done more is', 'said than done'])
print(f'Кол-во партиций: {rdd.getNumPartitions()}')

Кол-во партиций: 2


In [ ]:
rdd.take(3)

['After all is', 'said and', 'done more is']

In [ ]:
rdd.glom().collect()

[['After all is', 'said and'], ['done more is', 'said than done']]

In [ ]:
rdd.flatMap(lambda row: row.split(' ')).glom().collect()

[['After', 'all', 'is', 'said', 'and'],
 ['done', 'more', 'is', 'said', 'than', 'done']]

In [ ]:
rdd.flatMap(lambda row: row.split(' ')).map(lambda word: (word, 1)).collect()

[('After', 1),
 ('all', 1),
 ('is', 1),
 ('said', 1),
 ('and', 1),
 ('done', 1),
 ('more', 1),
 ('is', 1),
 ('said', 1),
 ('than', 1),
 ('done', 1)]

In [ ]:
rdd.flatMap(lambda row: row.split(' '))\
.map(lambda word: (word, 1))\
.reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False).collect()

[('is', 2),
 ('said', 2),
 ('done', 2),
 ('more', 1),
 ('than', 1),
 ('After', 1),
 ('all', 1),
 ('and', 1)]

In [ ]:
a = rdd.flatMap(lambda row: row.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)

In [ ]:
a.take(1)

[('is', 2)]

# Примеры RDD

 * In-Memory Processing
 * Immutability
 * Fault Tolerance
 * Lazy Evolution
 * Partitioning
 * Parallelize

## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [ ]:
# Вспомогательная функция

def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [ ]:
data = sc.textFile('/content/train.csv')

In [ ]:
data.take(3)

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2']

In [ ]:
data_header = data.first()
data_header

'battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range'

In [ ]:
data = data.filter(lambda row: row != data_header)
data.take(3)

['842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2',
 '563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2']

In [ ]:
data = data.map(lambda row: row.split(','))
beautiful_print(data, 3, 'Splitted:')

Splitted:
	 ['842', '0', '2.2', '0', '1', '0', '7', '0.6', '188', '2', '2', '20', '756', '2549', '9', '7', '19', '0', '0', '1', '1']
	 ['1021', '1', '0.5', '1', '0', '1', '53', '0.7', '136', '3', '6', '905', '1988', '2631', '17', '3', '7', '1', '1', '0', '2']
	 ['563', '1', '0.5', '1', '2', '1', '41', '0.9', '145', '5', '6', '1263', '1716', '2603', '11', '2', '9', '1', '1', '0', '2']


In [ ]:
data = data.map(lambda row: [float(el) for el in row])
beautiful_print(data, 3, 'Float:')

Float:
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0, 2.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0, 2.0]


In [ ]:
col_num = [x[0] for x in enumerate(data_header.split(',')) if x[1] == 'ram'][0]

In [ ]:
data.map(lambda row: (round(row[col_num] / 1024, 1), 1)).countByKey()

defaultdict(int,
            {0.2: 1,
             0.3: 57,
             0.4: 51,
             0.5: 56,
             0.6: 65,
             0.7: 56,
             0.8: 51,
             0.9: 49,
             1.0: 51,
             1.1: 44,
             1.2: 66,
             1.3: 61,
             1.4: 67,
             1.5: 36,
             1.6: 59,
             1.7: 48,
             1.8: 57,
             1.9: 41,
             2.0: 61,
             2.1: 55,
             2.2: 55,
             2.3: 76,
             2.4: 52,
             2.5: 61,
             2.6: 59,
             2.7: 49,
             2.8: 45,
             2.9: 53,
             3.0: 45,
             3.1: 54,
             3.2: 57,
             3.3: 49,
             3.4: 53,
             3.5: 59,
             3.6: 56,
             3.7: 53,
             3.8: 63,
             3.9: 29})